# Descriptions

* Newly generated images will have main object in the middle and four other copies in each corner.
* Newly generated images will have the same size as their original images.

In [1]:
# Load libraries
import os
import json 
import random
import urllib
import time

from pycocotools.coco import COCO
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import load_model
import PIL

import utils

%matplotlib inline

Using TensorFlow backend.


# Load image

In [2]:
# Define image directory
projectDir=os.getcwd()
dataDir='..'
dataType='val2017'
imageDir='{}/images/'.format(dataDir)
annFile='{}/images/{}_selected/annotations/instances_{}.json'.format(dataDir,dataType,dataType)

# initialize COCO api for instance annotations
coco=COCO(annFile); print()
obj_ids = coco.getAnnIds()
obj_anns = coco.loadAnns(obj_ids)
obj_anns_df = pd.DataFrame(obj_anns)

loading annotations into memory...
Done (t=0.62s)
creating index...
index created!



# Load model

In [3]:
# load yolov3 model
model = utils.YOLOv3()

Instructions for updating:
Colocations handled automatically by placer.
Model initialization completes.


# Crop method

```
--------------------------
|++                    ++|
|++   xx               ++|
|     xx                 |
|++                    ++|
|++                    ++|
--------------------------

+: copy of object
x: original object
```

In [18]:
def is_good_to_copy_object(bbox, img_sz):
    """Returns two parameters:
    parameter 1: bool -> can or cannot copy object to corner
    parameter 2: dict of BoundBox class -> coordinate of corner
    """
    img_w, img_h = img_sz
    # bbox_x, bbox_y, bbox_w, bbox_h = map(round,bbox)
    bbox_x, bbox_y, bbox_w, bbox_h = bbox
    bbox_class = utils.BoundBox(bbox_x, bbox_y, bbox_x+bbox_w, bbox_y+bbox_h)
    
    corners = ([0,0], [0,1], [1,0], [1,1])
    labels = ['top_L', 'top_R', 'bot_L', 'bot_R']
    corner_boxes = dict()
    for corner, label in zip(corners, labels):
        cor_x = corner[1]*(img_w-bbox_w)
        cor_y = corner[0]*(img_h-bbox_h)
        corner_box = utils.BoundBox(cor_x, cor_y, bbox_w, bbox_h)
        iou = utils.bbox_iou(corner_box, bbox_class)
        if iou > 0:
            return False, corner_boxes
        corner_boxes[label] = [cor_x, cor_y, bbox_w, bbox_h]
    return True, corner_boxes

## Copy object to corners

In [38]:
def generate_image_sample(img_path, object_bbox, target_bbox):
    img = utils.load_image(img_path, return_tensor=False)
    img_tensor = utils.convert_to_tensor(img)
    
    obj_x, obj_y, obj_w, obj_h = map(round,object_bbox)
    tar_x, tar_y, tar_w, tar_h = map(round,target_bbox)
    img_tensor[:,tar_x:tar_x+tar_w, tar_y:tar_y+tar_h] = img_tensor[:,obj_x:obj_x+obj_w, obj_y:obj_y+obj_h]

    fig, axs = plt.subplots(1, 2)
    axs[0].imshow(img)
    axs[1].imshow(np.squeeze(img_tensor))
    plt.show(True)

obj_id = random.choice(obj_ids)
obj_ann = coco.loadAnns(obj_id)[0]
img_ann = coco.loadImgs(obj_ann["image_id"])[0]
generate_image_sample(img_ann["coco_url"], obj_ann["bbox"], [0, 0, obj_ann["bbox"][2], obj_ann["bbox"][3]])

ValueError: could not broadcast input array from shape (1,77,156,3) into shape (1,102,156,3)

# Run prediction

In [20]:
count = 0
result_df = pd.DataFrame()
st = time.time()
for idx, obj_ann in obj_anns_df.iterrows():
    img_ann = coco.loadImgs(obj_ann["image_id"])[0]
    image_w, image_h = img_ann['width'], img_ann['height']
    
    is_good, corner_boxes = is_good_to_copy_object(obj_ann['bbox'], (image_w, image_h))
    if is_good:
        count += 1
        print(f"Qualified obj count: {count}")


        item = {
            "obj_id": obj_ann["id"],
            "coco_url": img_ann["coco_url"],
            "corner_area": corner_boxes,
            "true_class": obj_ann["category_id"],
            "true_bbox": obj_ann["bbox"],
        }
        result_df = result_df.append(item, ignore_index=True)
et = time.time()
print(f"total time = {et-st}")

j count: 29224
Qualified obj count: 29225
Qualified obj count: 29226
Qualified obj count: 29227
Qualified obj count: 29228
Qualified obj count: 29229
Qualified obj count: 29230
Qualified obj count: 29231
Qualified obj count: 29232
Qualified obj count: 29233
Qualified obj count: 29234
Qualified obj count: 29235
Qualified obj count: 29236
Qualified obj count: 29237
Qualified obj count: 29238
Qualified obj count: 29239
Qualified obj count: 29240
Qualified obj count: 29241
Qualified obj count: 29242
Qualified obj count: 29243
Qualified obj count: 29244
Qualified obj count: 29245
Qualified obj count: 29246
Qualified obj count: 29247
Qualified obj count: 29248
Qualified obj count: 29249
Qualified obj count: 29250
Qualified obj count: 29251
Qualified obj count: 29252
Qualified obj count: 29253
Qualified obj count: 29254
Qualified obj count: 29255
Qualified obj count: 29256
Qualified obj count: 29257
Qualified obj count: 29258
Qualified obj count: 29259
Qualified obj count: 29260
Qualified obj

In [21]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29963 entries, 0 to 29962
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   coco_url     29963 non-null  object 
 1   corner_area  29963 non-null  object 
 2   obj_id       29963 non-null  float64
 3   true_bbox    29963 non-null  object 
 4   true_class   29963 non-null  float64
dtypes: float64(2), object(3)
memory usage: 1.1+ MB


In [22]:
result_df.head()

,coco_url,corner_area,obj_id,true_bbox,true_class
0,http://images.cocodataset.org/val2017/00000028...,"{'top_L': [0.0, 0.0, 38.65, 28.67], 'top_R': [...",1768.0,"[473.07, 395.93, 38.65, 28.67]",18.0
1,http://images.cocodataset.org/val2017/00000006...,"{'top_L': [0.0, 0.0, 151.97, 279.77], 'top_R':...",1773.0,"[272.1, 200.23, 151.97, 279.77]",18.0
2,http://images.cocodataset.org/val2017/00000049...,"{'top_L': [0.0, 0.0, 154.53, 208.61], 'top_R':...",3658.0,"[0.0, 421.09, 154.53, 208.61]",18.0
3,http://images.cocodataset.org/val2017/00000055...,"{'top_L': [0.0, 0.0, 188.88, 285.91], 'top_R':...",4594.0,"[427.58, 77.87, 188.88, 285.91]",18.0
4,http://images.cocodataset.org/val2017/00000007...,"{'top_L': [0.0, 0.0, 170.45, 222.07], 'top_R':...",7544.0,"[197.97, 117.22, 170.45, 222.07]",18.0
